## Konfiguracja

In [7]:
from google.colab import userdata

In [8]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive/')

!ln -s /content/gdrive/My\ Drive/ ./mydrive

%cd /content/gdrive/MyDrive/projekt_inzynierski

/
Mounted at /content/gdrive/
/content/gdrive/MyDrive/projekt_inzynierski


In [9]:
!git config --global user.email "klaudiacayen@gmail.com"
!git config --global user.name "klaudia-pruchnik"

# TOKEN = userdata.get('TOKEN')
# REPO_URL = "https://github.com/klaudia-pruchnik/Proptify"
# !git clone https://{TOKEN}@{REPO_URL.replace('https://', '')}

REPO_NAME = "Proptify"
%cd /content/gdrive/MyDrive/projekt_inzynierski/{REPO_NAME}

!git add .

/content/gdrive/MyDrive/projekt_inzynierski/Proptify


In [4]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [10]:
!git add .
!git commit -m "add sampling final playlist"
!git push

[main d24c21d] add sampling final playlist
 2 files changed, 1 insertion(+), 739 deletions(-)
 delete mode 100644 algorithm.ipynb
 create mode 100644 algoritm_near_finished.ipynb
To https://github.com/klaudia-pruchnik/Proptify
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/klaudia-pruchnik/Proptify'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [ ]:
!pip install sentence-transformers -q
!pip install -q faiss-cpu sentence-transformers
!python -m spacy download en_core_web_sm
!python -m spacy download pl_core_news_sm
!pip install gliner
!python -m spacy download en_core_web_md
!python -m spacy download pl_core_news_lg
!pip install spacy gliner transformers torch
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 48.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 6.8 MB/s eta 0:00:00
   ━━━

In [ ]:
import faiss
import torch
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from math import log
from numpy.linalg import norm
import re
from gliner import GLiNER
import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
from langdetect import detect, DetectorFactory

Modele

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_e5 = SentenceTransformer('intfloat/multilingual-e5-base', device=device)
model_gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Dataframy

In [ ]:
df_songs = pd.read_parquet("../df_full_with_embeddings.parquet")

## Data preparation dodatkowe

zamień tag_list na listę

In [ ]:
def _ensure_list(x):
    if isinstance(x, list): return x
    if pd.isna(x) or x is None: return []
    return [t.strip() for t in str(x).split(",") if t.strip()]

df_songs["tags_list"] = df_songs["tags"].apply(_ensure_list)
df_songs["tag_count"] = df_songs["tags_list"].apply(len)
df_songs.head(5)

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,spotify_url,explicit,album_images,spotify_id,n_tempo,n_loudness,tags_list,tags_count,tags_embedding,tag_count
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,"rock, alternative, indie, alternative_rock, in...",None,2004,222200,0.355,0.918,...,https://open.spotify.com/track/003vvx7Niy0yvhv...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",003vvx7Niy0yvhvHt4a68B,0.619996,0.874265,"[rock, alternative, indie, alternative_rock, i...",6,"[-0.005010171327739954, 0.01898571476340294, -...",6
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,"rock, alternative, indie, pop, alternative_roc...",None,2006,258613,0.409,0.892,...,https://open.spotify.com/track/5qqabIl2vWzo9Ap...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",5qqabIl2vWzo9ApSC317sa,0.730137,0.874061,"[rock, alternative, indie, pop, alternative_ro...",9,"[0.003282089252024889, 0.02081618271768093, -0...",9
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,0.826,...,https://open.spotify.com/track/2RsAajgo0g7bMCH...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",2RsAajgo0g7bMCHxwH3Sk0,0.502363,0.851906,"[rock, alternative, alternative_rock, 90s, gru...",5,"[-0.000298114464385435, 0.017946673557162285, ...",5
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,"rock, alternative, indie, alternative_rock, in...",None,2004,237026,0.279,0.664,...,https://open.spotify.com/track/20I8RduZC2PWMWT...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",20I8RduZC2PWMWTDCZuuAN,0.437682,0.803699,"[rock, alternative, indie, alternative_rock, i...",8,"[0.0032091771718114614, 0.018290618434548378, ...",8
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,0.430,...,https://open.spotify.com/track/70LcF31zb1H0PyJ...,True,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",70LcF31zb1H0PyJoS1Sx1r,0.384441,0.786666,"[rock, alternative, indie, alternative_rock, i...",8,"[0.007342000957578421, 0.020590929314494133, -...",8


Uzupełnij tagi ich tagami nadrzędnymi dla lepszego dopasowania (to trzeba przenieść do innego pliku i poprawić bazę)

In [ ]:
# reguły: subgatunek -> nadrzędny
ALSO_ADD_PARENT = {
    # rock
    "progressive rock": "rock",
    "classic rock": "rock",
    "indie rock": "rock",
    "hard rock": "rock",
    "pop rock": "rock",
    "psychedelic rock": "rock",
    "punk rock": "rock",
    "blues rock": "rock",
    "post rock": "rock",

    # metal
    "heavy metal": "metal",
    "death metal": "metal",
    "black metal": "metal",
    "doom metal": "metal",
    "thrash metal": "metal",
    "melodic death metal": "metal",
    "symphonic metal": "metal",
    "gothic metal": "metal",
    "nu metal": "metal",
    "progressive metal": "metal",
    "power metal": "metal",
    "metalcore": "metal",

    # pop / electronic
    "indie pop": "pop",
    "synthpop": "pop",
    "drum and bass": "electronic",
}

# 2) reguły: słowa-klucze → nadrzędny
PARENT_KEYWORDS = {
    "rock": ["rock"],
    "metal": ["metal"],
    "pop": ["pop", "britpop"],
    "hip hop": ["hip hop", "rap"],
    "electronic": ["electronic", "techno", "house", "trance", "idm", "downtempo", "electro", "ambient"],
    "jazz": ["jazz"],
    "classical": ["classical"],
    "punk": ["punk"],
    "folk": ["folk"],
    "blues": ["blues"],
    "country": ["country"],
    "reggae": ["reggae"],
}

def expand_tags(tag_list):
    if tag_list is None or (isinstance(tag_list, float) and pd.isna(tag_list)):
        tag_list = []

    # normalizacja minimalna: małe litery, zamiana _ → spacja
    tags = {str(t).lower().replace("_", " ").strip() for t in tag_list}

    # 1) subgatunek → nadrzędny
    for child, parent in ALSO_ADD_PARENT.items():
        if child in tags:
            tags.add(parent)

    # 2) słowa kluczowe → nadrzędny
    for parent, kws in PARENT_KEYWORDS.items():
        if any(kw in t for t in tags for kw in kws):
            tags.add(parent)

    return sorted(tags)

# zastosowanie
df_songs["tags_list"] = df_songs["tags_list"].apply(expand_tags)


In [ ]:
df_songs["tags_list"] = df_songs["tags_list"].apply(
    lambda lst: [tag.replace("_", " ") for tag in lst]
)

# Algorytm dopasowania utworów

## Przygotowanie

Budowanie inverted index do późniejszego scorowania utworów

In [ ]:
df_songs = df_songs.reset_index(drop=True)

def build_inverted_index(df_songs: pd.DataFrame, tags_col: str = "tags_list"):
    inv = defaultdict(list)
    df_count = defaultdict(int)  # document frequency: w ilu utworach wystąpił tag

    for i, tags in enumerate(df_songs[tags_col]):
        if not tags:
            continue
        seen = set()
        for t in tags:
            inv[t].append(i)
            if t not in seen:
                df_count[t] += 1
                seen.add(t)

    # zamieniamy listy na numpy dla szybkości
    for t in inv:
        inv[t] = np.asarray(inv[t], dtype=np.int32)

    return inv, df_count

INV_INDEX, DF_COUNT = build_inverted_index(df_songs, tags_col="tags_list")
N_SONGS = len(df_songs)
AVG_TAG_LEN = float(df_songs["tag_count"].mean()) if "tag_count" in df_songs else 10.0

Config do wszystkich parametrów algorytmu

In [ ]:
RETRIEVAL_CONFIG = {
    "n_candidates": 200,    # początkowe top najlepiej dopasowanych wg FAISS
}

SCORE_TIERS_CONFIG = {
    "t_high": 0.9,
    "t_mid": 0.7,
    "min_final": 100,         # chcemy finalnie tyle
    "max_c_from_low_tier": 15,  # ile max brać z Tier C gdy brakuje
}

POPULARITY_CONFIG = {
    "p_high": 70,
    "p_mid": 35,
    # procentowy miks w finalnym secie (docelowy)
    "mix": {
        "high": 0.4,     # popularne
        "mid": 0.35,     # średnie
        "low": 0.25,     # niszowe
    },
    "forced_popular": 2,    # ile utworów bardzo popularnych wstawić na sztywno
    "forced_popular_min": 80,
}

SAMPLING_CONFIG = {
    "final_n": 15,
    "alpha": 2.0,   # jak mocno faworyzujemy wyższy score przy losowaniu
    "do_shuffle": True,
}

QUERY_TAGS_CONFIG = {
    "rel_keep": 0.98,   # próg względny: bierzemy tagi >= 70% najlepszego
    "abs_keep": 0.9,  # próg absolutny: odetnij totalny szum
    "min_keep": 1,     # min liczba tagów w profilu
    "max_keep": 12,    # max liczba tagów w profilu
    "top_m_per_ngram": 2,
    "min_ngram_sim": 0.7,
    "min_unigram_sim": 0.5,
    "power": 1,
    "include_unigrams": True,
}

TAG_SCORING_CONFIG = {
    "use_idf": False,           # waż tagi rzadkie wyżej
    "k1": 1.2,                 # siła normalizacji „długości dokumentu” (liczby tagów utworu)
    "b": 0.75,
    "len_norm": False,
    "query_pow": 1.0,          # możesz podnieść np. 1.2 by ostrzej różnicować wagi tagów z promptu
    "normalize_by_tags": False # alternatywna, prostsza normalizacja: score / sqrt(n_tags_utworu)
}


## Wyciąganie relevant phrases z prompta

In [ ]:
# Ustawiamy ziarno dla detekcji języka (żeby wyniki były powtarzalne dla krótkich tekstów)
DetectorFactory.seed = 0

In [ ]:
# spaCy: Model do parsowania gramatycznego i Matchera (zapewnia precyzyjne wzorce)
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")

In [ ]:
GENERIC_LEMMAS = [
    # Polski
    "muzyka", "utwór", "piosenka", "kawałek", "playlista", "lista", "numer", "rok", "klimat", "styl",
    # Angielski
    "music", "song", "track", "playlist", "list", "number", "vibe", "tune", "genre", "style"
]

GENERIC_VERBS = [
    # --- POLSKI (Bezokoliczniki / Lematy) ---
    # Szukanie / Chcenie
    "szukać", "poszukiwać", "chcieć", "pragnąć", "potrzebować", "woleć", "wymagać",
    # Bycie / Posiadanie
    "być", "mieć", "znajdować", "znaleźć",
    # Słuchanie / Odtwarzanie
    "słuchać", "posłuchać", "usłyszeć", "grać", "zagrać", "puszczać", "puścić", "odtworzyć", "zapodać",
    # Prośby / Rekomendacje
    "prosić", "polecić", "polecać", "rekomendować", "sugerować", "zaproponować", "dawać", "dać",

    # --- ANGIELSKI (Base forms) ---
    # Searching / Wanting
    "search", "look", "find", "want", "need", "desire", "wish", "require",
    # Being / Having
    "be", "have", "get",
    # Listening / Playing
    "listen", "hear", "play", "replay", "stream",
    # Requests
    "give", "recommend", "suggest", "show", "provide",
]

NEGATION_TERMS = [
    # PL
    "nie", "bez", "mało", "zero", "ani", "żaden", "brak", "mniej",
    # EN
    "no", "not", "without", "less", "non", "neither", "nor", "lack", "zero"
]

In [ ]:
def create_matcher_for_nlp(nlp_instance):
    """Tworzy obiekt Matcher przypisany do konkretnego modelu językowego"""
    matcher = Matcher(nlp_instance.vocab)

    # Warunek wykluczający
    # To musi być Rzeczownik, ale NIE MOŻE być na liście generycznej
    noun_filter = {
        "POS": {"IN": ["NOUN", "PROPN"]},
        "IS_STOP": False,
        "LEMMA": {"NOT_IN": GENERIC_LEMMAS}
    }

    matcher.add("FRAZA", [
        # 1. Samodzielne NOUN/PROPN (rock)
        [noun_filter],
        # 2. ADJ + NOUN (szybki bas)
        [{"POS": "ADJ"}, noun_filter],
        # 3. (ADV)? + ADP + NOUN (z tańca, z klimatem -> filtrowane)
        [{"POS": "ADV", "OP": "?"}, {"POS": "ADP"}, noun_filter],
        # 4. ADV + ADJ (bardzo wesoła)
        [{"POS": "ADV"}, {"POS": "ADJ", "IS_STOP": False}],
        # 5. ADJ Samotny (rockowa)
        [{"POS": "ADJ", "IS_STOP": False}],
        # 6. Złożone Rzeczowniki (post rock)
        [{"POS": {"IN": ["NOUN", "PROPN"]}, "IS_STOP": False}, noun_filter],
        # 7. Złożona relacja (dobra do tańca)
        [{"POS": "ADV", "OP": "?"}, {"POS": "ADJ"}, {"POS": "ADP"}, noun_filter],
        # 8. Celuje w: Czasownik opisowy + Rzeczownik/Adjektive/Zaimek
        [
            {"POS": "VERB", "LEMMA": {"NOT_IN": GENERIC_VERBS}},
            {"POS": {"IN": ["NOUN", "ADJ", "PRON"]}, "OP": "+"} # Obiekt czasownika (może być więcej niż jedno słowo)
        ]
    ])
    return matcher

# Tworzymy matchery raz na starcie
matcher_pl = create_matcher_for_nlp(nlp_pl)
matcher_en = create_matcher_for_nlp(nlp_en)


In [ ]:
# !git commit -m "get features for phrases"
# !git push

In [ ]:
# 4. FUNKCJA POMOCNICZA: SPRAWDZANIE NEGACJI
# ----------------------------------------------------------------------

def is_span_negated(doc, start_index, window=2):
    """
    Sprawdza, czy przed frazą (start_index) stoi słowo przeczące.
    Patrzy 'window' tokenów wstecz.
    """
    lookback = max(0, start_index - window)
    preceding_tokens = doc[lookback:start_index]

    for token in preceding_tokens:
        if token.text.lower() in NEGATION_TERMS:
            return True
    return False

TODO: stąd możemy wyciągnąc sprawdzenie jaki to język (żeby móc tę informację wykorzystać niżej - do dodania prefiksu "muzyka" do funkcji dopasowującej cechy audio e5)

In [ ]:
# ==============================================================================
# GŁÓWNA FUNKCJA EKSTRAKCJI
# ==============================================================================

def extract_relevant_phrases(prompt):
    # 1. Setup językowy
    prompt = prompt.lower()
    prompt_clean = prompt.strip()

    if not prompt_clean:
        return []

    try:
        lang_code = detect(prompt)
    except:
        lang_code = 'pl'

    if lang_code == 'en':
        current_nlp = nlp_en
        current_matcher = matcher_en
        lang_msg = "EN"
    else:
        current_nlp = nlp_pl
        current_matcher = matcher_pl
        lang_msg = "PL"

    doc = current_nlp(prompt)

    # 2. Matcher (Z Negacją)
    matcher_matches = current_matcher(doc)

    # Konwersja matchy na obiekty Span
    matcher_spans = [doc[start:end] for match_id, start, end in matcher_matches]

    # filter_spans usuwa nakładające się frazy (np. "rock" wewnątrz "post rock")
    # zostawiając najdłuższe dopasowanie
    combined_spans = filter_spans(matcher_spans)

    final_phrases = []

    for span in combined_spans:
        # A. Sprawdzanie negacji
        if is_span_negated(doc, span.start):
            # print(f"Odrzucono (negacja): '{span.text}'")
            continue

        final_phrases.append(span.text.lower())

    # 3. Czyszczenie i sortowanie wyników
    unique_phrases = sorted(list(set([p.strip() for p in final_phrases if len(p.strip()) > 2])))

    print(f"[{lang_msg}] Prompt: '{prompt}' \n-> {unique_phrases}")

    return unique_phrases

In [ ]:
# ----------------------------------------------------------------------
# 4. PRZYKŁADY UŻYCIA
# ----------------------------------------------------------------------

TEST_PROMPT_1 = "szukam muzyki rockowej, ale takiej pełnej spokoju i bardzo wesołej, trochę do tańca"
TEST_PROMPT_2 = "rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii"
TEST_PROMPT_3 = "rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii, post rock, alternative rock, rock alternatywny, pop"
TEST_PROMPT_4 = "muzyka rockowa z lat 90, z klimatem podróży, postpankowa"
TEST_PROMPT_5 = "muzyka bez słów, smutna, nostalgiczna, spokojna"
TEST_PROMPT_6 = "szybka, intensywna, bardzo dobra do tańca, zabawy, zajsta do tańca, idealna do tańca, "
TEST_PROMPT_7 = "albo zwykły rock, albo jakiś post rock albo punk, coś takiego"
TEST_PROMPT_8 = "muzyka dynamiczna, szybko, szybkie tempo, wysokie tempo, energiczna"
TEST_PROMPT_9 = "muzyka, która koi nerwy"
TEST_PROMPT_10 = "muzyka, która koi nerwy"
TEST_PROMPT_11 = "Szukam muzyki rockowej, ale nie smutnej"
TEST_PROMPT_12 = "I want energetic songs, no slow music"
TEST_PROMPT_13 = "Chcę wesołe piosenki, coś szybkiego, ruchliwego, rock, muzyka popowa, z lat 90, dużo gitary, shoegaze, "
# TEST_PROMPT_14 = "Vaporwave, J-Core (Japanese Hardcore), Blackened Death-Doom, Skweee, Dungeon Synth, Breakcore, Mathcore, Drone Metal (Ambient Drone), Folktronica, Lowercase"
TEST_PROMPT_14 = "szybki rock"

picked_prompt = TEST_PROMPT_3

relevant_phrases = extract_relevant_phrases(picked_prompt)
# extract_and_route(TEST_PROMPT_4)

[PL] Prompt: 'rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii, post rock, alternative rock, rock alternatywny, pop' 
-> ['alternative rock', 'alternatywny', 'dance', 'do tańca', 'maksymalnej energii', 'pop', 'post rock', 'rock', 'zależy mi']


## Dzielimy znalezione frazy na audio i tag

In [ ]:
# ==============================================================================
# KONFIGURACJA BRAMKARZA (ROUTERA)
# ==============================================================================

LABELS_CONFIG = {
    # --- GRUPA 1: TAGI (Konkrety do filtrowania bazy) ---
    "gatunek_muzyczny": {
        "desc": "rock, pop, jazz, hip hop, metal, indie, alternative, emo, psychedelic, industrial, grunge, punk, folk, electronic, experimental, noise music",
        "route": "TAGS"
    },
    "klimat_styl": {
        "desc": "chill, chillout, mellow, ambient, lounge, dark, beautiful, love",
        "route": "TAGS"
    },
    "typ_utworu": {
        "desc": "soundtrack, ost, muzyka filmowa, ścieżka dźwiękowa, remix",
        "route": "TAGS"
    },
    "instrument": {
        "desc": "piano, guitar, drums, violin, bass, saxophone, synthesizer, vocals",
        "route": "TAGS"
    },
    "okres_czasu": {
        "desc": "80s, 90s, 00s, 2020s, oldies, retro, klasyk, lata 90, lata 80",
        "route": "TAGS"
    },
    "pochodzenie": {
        "desc": "polish, american, british, french, k-pop, latino, spanish, deutsch",
        "route": "TAGS"
    },

    # --- GRUPA 2: WSZYSTKO INNE (Wsad dla modelu E5) ---
    # Zamiast rozbijać na valence/tempo, wrzucamy wszystko co jest opisem tutaj.
    # GLiNER ma tylko odróżnić "Rock" (Tag) od "Szybka" (Audio).
    "cecha_audio": {
        "desc": "sad, happy, fast, slow, danceable, party, energetic, calm, relaxing, loud, quiet, acoustic, electronic, melancholic, gloomy, euphoric, club banger",
        "route": "AUDIO"
    }
}

# Generowanie zmiennych
GLINER_LABELS = [f"{k} ({v['desc']})" for k, v in LABELS_CONFIG.items()]
ROUTING_MAP = {k: v['route'] for k, v in LABELS_CONFIG.items()}

In [ ]:
def get_label_config_lists(config):
    """
    Tworzy listę etykiet dla GLiNERa (klucz + opis) oraz mapę powrotną.
    """
    gliner_labels = []
    label_mapping = {} # Mapa "nazwa (opis)" -> "nazwa"

    for key, value in config.items():
        full_label = f"{key} ({value['desc']})"
        gliner_labels.append(full_label)
        label_mapping[full_label] = key

    return gliner_labels, label_mapping

GLINER_LABELS_LIST, GLINER_LABEL_MAP = get_label_config_lists(LABELS_CONFIG)

def classify_phrases_with_gliner(prompt, spacy_phrases, model, threshold=0.3):
    """
    Klasyfikuje frazy wykryte przez spaCy używając GLiNERa uruchomionego na całym prompcie.
    """
    if not spacy_phrases:
        return []

    # 1. Uruchamiamy GLiNER na całym tekście (kontekst jest kluczowy)
    #    Dzięki temu odróżni "Rock" (gatunek) od "Szybka" (audio)
    gliner_predictions = model.predict_entities(prompt, GLINER_LABELS_LIST, threshold=threshold)

    results = []

    # 2. Iterujemy po frazach ze spaCy i szukamy dla nich etykiety w wynikach GLiNERa
    for phrase in spacy_phrases:
        matched_category = None
        matched_route = "AUDIO" # Domyślny routing (bezpieczny fallback)

        # Normalizacja frazy spaCy do porównania
        phrase_clean = phrase.lower().strip()

        # Szukamy czy ta fraza została też znaleziona przez GLiNERa
        # Sprawdzamy czy tekst encji GLiNERa zawiera się w frazie spaCy lub odwrotnie
        best_score = 0

        for entity in gliner_predictions:
            entity_text = entity['text'].lower().strip()

            # Sprawdzenie pokrycia (overlap)
            if phrase_clean in entity_text or entity_text in phrase_clean:
                # Jeśli mamy dopasowanie, pobieramy czystą nazwę kategorii
                full_label = entity['label']
                short_key = GLINER_LABEL_MAP.get(full_label)

                if short_key:
                    matched_category = short_key
                    # Pobieramy routing z konfiguracji
                    matched_route = LABELS_CONFIG[short_key]['route']
                    break # Znaleźliśmy etykietę, idziemy do następnej frazy spaCy

        # Jeśli GLiNER nic nie znalazł dla tej frazy, ale spaCy ją wykryło:
        # Opcja A: Oznaczamy jako ogólną "cecha_audio" (bezpieczne dla wektorów E5)
        # Opcja B: Oznaczamy jako "unknown"
        if not matched_category:
            matched_category = "cecha_audio" # Fallback
            matched_route = "AUDIO"

        results.append({
            "phrase": phrase,
            "category": matched_category,
            "route": matched_route
        })

    return results

In [ ]:
classified_phrases = classify_phrases_with_gliner(picked_prompt, relevant_phrases, model_gliner)
classified_phrases

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'phrase': 'alternative rock',
  'category': 'gatunek_muzyczny',
  'route': 'TAGS'},
 {'phrase': 'alternatywny', 'category': 'gatunek_muzyczny', 'route': 'TAGS'},
 {'phrase': 'dance', 'category': 'cecha_audio', 'route': 'AUDIO'},
 {'phrase': 'do tańca', 'category': 'cecha_audio', 'route': 'AUDIO'},
 {'phrase': 'maksymalnej energii',
  'category': 'cecha_audio',
  'route': 'AUDIO'},
 {'phrase': 'pop', 'category': 'gatunek_muzyczny', 'route': 'TAGS'},
 {'phrase': 'post rock', 'category': 'gatunek_muzyczny', 'route': 'TAGS'},
 {'phrase': 'rock', 'category': 'gatunek_muzyczny', 'route': 'TAGS'},
 {'phrase': 'zależy mi', 'category': 'cecha_audio', 'route': 'AUDIO'}]

BŁĘDY:
* nie rozpoznaje 'z lat 90'
* nie rozponaje rzeczy jak "j-core", "death-doom" - z "-"
* szybkie vaporwave, szybki rock itp. tutaj lepiej by było to rozdzielić na 2
* rozpoznało "zależy mi" jako audio feature ??

TODO:
* można też na typ etapie podczas dzielenia na audio i tag dołożyć kategorię "trash"
* trzeba coś zrobić z wyciąganiem do czego ma służyć muzyka i zdefiniować, że np do biegania ma być energiczna, do gotowania itp, a do skupienia muzyka z np wysokim instrumentallness, raczej niską energią itp.


* 'fast, energetic': bieganie, jazda samochodem (loundness wysoki), gotowanie,
* 'focus': czytanie, pisanie, programowanie
* 'relax':
* itp.

In [ ]:
def prepare_queries_for_e5_separated(classified_data, original_prompt):
    """
    Rozdziela frazy na Tagi i Audio.
    Dla Audio: dodaje kontekstowy suffix (np. 'szybka' -> 'szybka muzyka'),
    aby poprawić jakość embeddingu w E5.
    """

    tags_queries = []
    audio_queries = []

    for item in classified_data:
        phrase = item['phrase']
        route = item['route']

        if route == 'TAGS':
            # Tagi zostawiamy "czyste" do mapowania (np. 'rock', 'gitara')
            tags_queries.append(phrase)

        elif route == 'AUDIO':
            # Audio modyfikujemy, aby E5 wiedziało, o co chodzi
            audio_queries.append(phrase)

    return {
        "tags_to_match": tags_queries,
        "audio_features": audio_queries
    }

In [ ]:
e5_queries_separated = prepare_queries_for_e5_separated(classified_phrases, picked_prompt)
e5_queries_separated

{'tags_to_match': ['alternative rock',
  'alternatywny',
  'pop',
  'post rock',
  'rock'],
 'audio_features': ['dance', 'do tańca', 'maksymalnej energii', 'zależy mi']}

## Dopasowanie audio

In [ ]:
FEATURE_DESCRIPTIONS = {
    'valence': [
        (0.1, "very low valence, very sad, melancholic, dark, gloomy emotional mood music"),
        (0.3, "low valence, bittersweet, thoughtful, introspective, moody emotional mood music"),
        # (0.5, "medium valence, neutral emotional mood, neither clearly happy nor clearly sad music"),
        (0.7, "high valence, positive, pleasant, warm, cheerful, uplifting emotional mood music"),
        (0.9, "very high valence, very happy, joyful, exstatic, euphoric, bright, feel-good emotional mood music")
    ],

    'danceability': [
        (0.1, "very low danceability, not danceable, abstract or experimental, weak or irregular rhythm music"),
        (0.3, "low danceability, little groove, minimal rhythm, not primarily for dancing music"),
        # (0.5, "medium danceability, some groove, steady rhythm music"),
        (0.7, "high danceability, clear beat, strong groove, good for dancing, club-oriented music"),
        (0.9, "very high danceability, strong groove, infectious rhythm, perfect for dancing, party, club banger music")
    ],

    'acousticness': [
        (0.1, "very low acousticness, fully electronic, synthetic, digital, computer-generated sound music"),
        (0.3, "low acousticness, mostly electronic with some subtle organic or acoustic elements music"),
        # (0.5, "medium acousticness, balanced mix of acoustic and electronic instruments, hybrid sound music"),
        (0.7, "high acousticness, mostly acoustic, organic, live instruments such as accoustic guitar or piano music"),
        (0.9, "very high acousticness, fully acoustic, unplugged, natural, organic instruments only music")
    ],

    'n_tempo': [
        (0.1, "very slow tempo, very slow pace, dragging rhythm music"),
        (0.3, "slow tempo, downtempo, slow pace, relaxed rhythm music"),
        (0.5, "medium tempo, moderate pace, walking pace music"),
        (0.7, "fast tempo, uptempo, quick pace, energetic rhythm music"),
        (0.9, "very fast tempo, rapid pace, racing rhythm, frantic speed music")
    ],

    'instrumentalness': [
        (0.1, "very low instrumentalness, strong presence of vocals and lyrics, clear singing, vocal-focused track"),
        (0.5, "medium instrumentalness, mix of vocals and instrumental sections, vocals present but not constant"),
        (0.9, "very high instrumentalness, fully instrumental track, no vocals, no singing, music without lyrics")
    ],

    'energy': [
        # Słowa kluczowe: Calm -> Active -> Hyperactive (Unikamy słów o głośności)
        (0.1, "very low energy, motionless, static, sleep-inducing, minimal activity music"),
        (0.3, "low energy, relaxed, laid-back, mellow, slow-moving atmosphere music"),
        (0.5, "medium energy, moderate pace, steady rhythm, balanced activity music"),
        (0.7, "high energy, active, driving rhythm, fast-paced, stimulating, busy arrangement music"),
        (0.9, "very high energy, hyperactive, restless, frantic, adrenaline-pumping, non-stop action music")
    ],

    'n_loudness': [
        # Słowa kluczowe: Silence -> Volume -> Noise (Fizyka dźwięku)
        (0.1, "very low loudness, barely audible, near silence, whisper-like volume, extremely quiet music"),
        (0.3, "low loudness, soft volume, background level, reduced amplitude, delicate sound music"),
        (0.5, "medium loudness, standard volume, normal mastering level music"),
        (0.7, "high loudness, loud volume, amplified sound, noisy, high amplitude music"),
        (0.9, "very high loudness, maximum volume, deafening, high decibels music")
    ],

    'speechiness': [
        (0.1, "very low speechiness, purely musical track, no spoken words, fully melodic music"),
        (0.3, "low speechiness, mostly music with occasional spoken words or short background phrases"),
        (0.5, "medium speechiness, balanced mix of speech and music, frequent spoken segments, rap-like or talky structure"),
    ],

    'noise': [
        # Nouns (Generic terms)
        (None, "music song track playlist list recording audio sound genre style vibe type kind number piece"),
        # TODO: przetestować, czy to się nie będzie myliło z instrumentallness

        # Verbs (Search intent)
        (None, "I am looking for I want I need search find play listen to give me recommend show me"),

        # Adjectives (Empty fillers)
        (None, "good very good nice great best cool amazing awesome some any kind of such a")
    ]
}

In [ ]:
def prepare_feature_embeddings():
    """Embeduj wszystkie opisy jako 'passages' (raz przy starcie)"""

    feature_embeddings = {}

    for feature_name, descriptions in FEATURE_DESCRIPTIONS.items():
        # E5 wymaga "passage: " dla dokumentów
        passages = [f"passage: {desc}" for value, desc in descriptions]
        embeddings = model_e5.encode(passages, normalize_embeddings=True)
        feature_embeddings[feature_name] = embeddings

    return feature_embeddings

# Setup (raz)
feature_embeddings = prepare_feature_embeddings()

In [ ]:
def phrases_to_features(phrases_list, confidence_threshold=0.78, lang_code='pl'):
    """
    Logika KONKURENCYJNA:
    1. Sprawdź surową frazę (bez "muzyka") ze wszystkimi kategoriami (w tym Noise).
       Jeśli wygra Noise -> odrzuć.
    2. Dla pozostałych dodaj "muzyka", aby precyzyjnie dopasować cechę.
    """
    if not phrases_list:
        return []

    # ==================================================================
    # ETAP 1: ELIMINACJA (Surowe frazy vs Wszystkie kategorie)
    # ==================================================================

    # Kodujemy frazy BEZ słowa "muzyka"
    raw_queries = [f"query: {p}" for p in phrases_list]
    raw_embeddings = model_e5.encode(raw_queries, normalize_embeddings=True)

    valid_indices = [] # Indeksy fraz, które przeszły selekcję

    for i, phrase in enumerate(phrases_list):
        current_raw_emb = raw_embeddings[i].reshape(1, -1)

        # Szukamy zwycięzcy dla surowej frazy
        best_cat_raw = None
        best_score_raw = -1.0

        for feature_name, _ in FEATURE_DESCRIPTIONS.items():
            target_embs = feature_embeddings[feature_name]

            # Porównanie
            sims = cosine_similarity(current_raw_emb, target_embs)[0]
            max_score = float(np.max(sims))

            if max_score > best_score_raw:
                best_score_raw = max_score
                best_cat_raw = feature_name

        # DECYZJA: Czy wygrał Noise?
        if best_cat_raw == 'noise':
            # print(f" 🗑️ Odrzucono (wygrał Noise): '{phrase}' (score: {best_score_raw:.3f})")
            continue # Fraza odpada
        else:
            # Fraza przeszła, zapamiętujemy jej indeks
            valid_indices.append(i)

    if not valid_indices:
        return []

    # ==================================================================
    # ETAP 2: PRECYZYJNE DOPASOWANIE (Z kontekstem "Muzyka")
    # ==================================================================

    # Wyciągamy frazy, które przetrwały
    clean_phrases = [phrases_list[i] for i in valid_indices]

    # Dodajemy kontekst (suffix)
    suffix = " muzyka" if lang_code == 'pl' else " music"
    context_queries = [f"query: {suffix} {p}" for p in clean_phrases]
    context_embeddings = model_e5.encode(context_queries, normalize_embeddings=True)

    found_features = {}

    for i, phrase in enumerate(clean_phrases):
        current_emb = context_embeddings[i].reshape(1, -1)

        phrase_best_cat = None
        phrase_best_val = None
        phrase_best_score = -1.0
        phrase_best_desc = ""

        # Iterujemy po cechach (już BEZ Noise, bo go wyeliminowaliśmy)
        for feature_name, descriptions in FEATURE_DESCRIPTIONS.items():
            if feature_name == 'noise':
                continue

            target_embs = feature_embeddings[feature_name]
            sims = cosine_similarity(current_emb, target_embs)[0]

            max_idx = int(np.argmax(sims))
            max_score = float(sims[max_idx])

            if max_score > phrase_best_score:
                phrase_best_score = max_score
                phrase_best_cat = feature_name
                phrase_best_val = descriptions[max_idx][0]
                phrase_best_desc = descriptions[max_idx][1]

        # Zapisujemy wynik (jeśli przekroczył próg jakości)
        if phrase_best_score >= confidence_threshold:
            print(f"   🎯 Fraza: '{phrase}' -> {phrase_best_cat} ({phrase_best_score:.3f})")

            # Rozwiązywanie konfliktów (Max Pooling)
            if phrase_best_cat in found_features:
                if phrase_best_score > found_features[phrase_best_cat]['confidence']:
                    found_features[phrase_best_cat] = {
                        'value': phrase_best_val,
                        'confidence': phrase_best_score,
                        # 'matched_description': phrase_best_desc
                    }
            else:
                found_features[phrase_best_cat] = {
                    'value': phrase_best_val,
                    'confidence': phrase_best_score,
                    # 'matched_description': phrase_best_desc
                }

    sorted_features = sorted(
        found_features.items(),
        key=lambda x: x[1]['confidence'],
        reverse=True
    )

    return sorted_features

In [ ]:
# phrases = ["popowa", "rock", "szybka", "fast"] # Przykładowy wynik z extract_relevant_phrases
# criteria = map_phrases_to_criteria(phrases, tag_threshold=0.8)
# print(criteria)

In [ ]:
# df_songs['tags_list'].explode().unique()

Tutaj nie działa dobrze - wszystko mi dało na medium dla:


`['alternatywny muzyka', 'dance muzyka', 'do tańca muzyka', 'maksymalnej energii muzyka', 'zależy mi muzyka']`

Wynik: `{'danceability': 0.5, 'n_tempo': 0.5, 'energy': 0.5, 'n_loudness': 0.5}`

In [ ]:
audio_features_queries = e5_queries_separated['audio_features']
print(audio_features_queries)
criteria_audio = phrases_to_features(audio_features_queries)
criteria_audio

['dance', 'do tańca', 'maksymalnej energii', 'zależy mi']
   🎯 Fraza: 'dance' -> danceability (0.820)
   🎯 Fraza: 'do tańca' -> danceability (0.814)
   🎯 Fraza: 'maksymalnej energii' -> energy (0.821)


[('energy', {'value': 0.9, 'confidence': 0.8213170766830444}),
 ('danceability', {'value': 0.9, 'confidence': 0.8195251226425171})]

## Dopasowanie tagów

In [ ]:
df_tag_embeddings = pd.read_parquet("../df_unique_tag_embeddings.parquet")
df_tag_embeddings

,tag,tag_embedding
0,cover,"[0.00831775926053524, 0.012594068422913551, -0..."
1,rnb,"[0.002914982382208109, 0.03465382382273674, -0..."
2,american,"[-0.0076368385925889015, 0.005218952428549528,..."
3,grunge,"[-0.010997523553669453, 0.02025771327316761, -..."
4,hip hop,"[0.024621689692139626, 0.01388248149305582, -0..."
...,...,...
95,swedish,"[0.014969518408179283, 0.014814469963312149, -..."
96,60s,"[0.002923240652307868, 0.019455421715974808, -..."
97,post hardcore,"[0.02131279744207859, 0.02616158500313759, -0...."
98,house,"[0.016028674319386482, 0.011981630697846413, -..."


Embeddingi tagów

In [ ]:
TAG_VECS = np.array(df_tag_embeddings["tag_embedding"].to_list(), dtype=np.float32)
TAGS = df_tag_embeddings["tag"].tolist()

In [ ]:
def map_phrases_to_tags(
    phrases: list[str],
    model,
    tag_vecs,          # Macierz embeddingów tagów (np. numpy array)
    tags_list,         # Lista nazw tagów odpowiadająca wierszom macierzy
    threshold: float = 0.65 # Minimalne dopasowanie, żeby w ogóle uznać tag
):
    """
    Dla każdej frazy znajduje DOKŁADNIE JEDEN, najlepiej pasujący tag z bazy.
    """
    if not phrases:
        return {}

    print(f"🏷️ Mapowanie 1:1 dla fraz: {phrases}")

    # 1. Batch Encoding (Szybkie wektoryzowanie zapytań)
    # Dodajemy prefix 'query:' zgodnie ze standardem E5
    q_vecs = model.encode(
        [f"query: {p}" for p in phrases],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")

    # 2. Obliczenie macierzy podobieństwa (Tags x Phrases)
    # Wynik to macierz o wymiarach [Liczba_Tagów, Liczba_Fraz]
    # Skalowanie (dot + 1) / 2 daje wynik w zakresie 0.0 - 1.0
    sims_matrix = (tag_vecs @ q_vecs.T + 1.0) / 2.0

    found_tags = {}

    # 3. Iteracja po każdej frazie (kolumnie macierzy)
    for i, phrase in enumerate(phrases):
        # Pobieramy kolumnę wyników dla bieżącej frazy
        phrase_scores = sims_matrix[:, i]

        # Znajdujemy indeks najlepszego taga (argmax)
        best_idx = np.argmax(phrase_scores)
        best_score = float(phrase_scores[best_idx])
        best_tag_name = tags_list[best_idx]

        # Sprawdzamy, czy wynik jest wystarczająco dobry
        if best_score >= threshold:
            print(f"   ✅ Fraza '{phrase}' -> Tag '{best_tag_name}' ({best_score:.3f})")

            # Zapisujemy wynik.
            # Jeśli inny tag już był zapisany (np. inna fraza też wskazała na "Rock"),
            # bierzemy ten z wyższym scorem (Max Pooling)
            if best_tag_name in found_tags:
                found_tags[best_tag_name] = max(found_tags[best_tag_name], best_score)
            else:
                found_tags[best_tag_name] = best_score
        else:
            # Opcjonalnie: logowanie, że nic nie pasowało
            # print(f"   ❌ Fraza '{phrase}' nie pasuje do żadnego tagu (max: {best_score:.3f})")
            pass

    return found_tags

In [ ]:
tags_queries = e5_queries_separated['tags_to_match']
print(tags_queries)
criteria_tags = map_phrases_to_tags(tags_queries, model=model_e5, tag_vecs=TAG_VECS, tags_list=TAGS, threshold=0.70)
criteria_tags

['alternative rock', 'alternatywny', 'pop', 'post rock', 'rock']
🏷️ Mapowanie 1:1 dla fraz: ['alternative rock', 'alternatywny', 'pop', 'post rock', 'rock']
   ✅ Fraza 'alternative rock' -> Tag 'alternative rock' (0.946)
   ✅ Fraza 'alternatywny' -> Tag 'alternative' (0.942)
   ✅ Fraza 'pop' -> Tag 'pop' (0.955)
   ✅ Fraza 'post rock' -> Tag 'post rock' (0.963)
   ✅ Fraza 'rock' -> Tag 'rock' (0.951)


{'alternative rock': 0.9458271861076355,
 'alternative': 0.9416692852973938,
 'pop': 0.9546395540237427,
 'post rock': 0.9630432724952698,
 'rock': 0.9506984353065491}

In [ ]:
# TAGS

## Algorytm

uzyskujemy wagi wsyztskich tagów zsumowane do 1

In [ ]:
def get_query_tag_weights(
    raw_tags
) -> dict[str, float]:
    """
    Normalizuje wagi tak, by sumowały się do 1.
    """
    s = sum(raw_tags.values())
    if s <= 0:
        return raw_tags

    return {t: v / s for t, v in raw_tags.items()}

In [ ]:
query_tag_weights = get_query_tag_weights(criteria_tags)
query_tag_weights

{'alternative rock': 0.19887542093416063,
 'alternative': 0.19800115522686768,
 'pop': 0.20072836342141864,
 'post rock': 0.20249538077192958,
 'rock': 0.19989967964562347}

In [ ]:
def score_songs_by_tags(
    query_tag_weights: dict[str, float],
    inv_index: dict[str, np.ndarray],
    song_tag_count: np.ndarray,
    idf_map: dict[str, float],
    cfg: dict,
) -> np.ndarray:
    """
    Liczy score dla wszystkich utworów na podstawie:
    - wag tagów z promptu,
    - odwróconego indeksu (tag -> lista indeksów utworów),
    - opcjonalnie IDF i normalizacji długości.

    Zwraca: wektor shape [n_songs], wartości w ~[0,1].
    """
    n_songs = song_tag_count.shape[0]

    use_idf        = cfg.get("use_idf", True)
    k1             = cfg.get("k1", 1.2)
    b              = cfg.get("b", 0.75)
    len_norm       = cfg.get("len_norm", True)
    query_pow      = cfg.get("query_pow", 1.0)
    normalize_tags = cfg.get("normalize_by_tags", False)

    # 1) wyostrz wagi zapytania, jeśli chcesz
    qtw = {}
    for t, w in query_tag_weights.items():
        w2 = w ** query_pow
        qtw[t] = w2

    s = sum(qtw.values())
    if s > 0:
        for t in qtw:
            qtw[t] /= s

    # 2) coverage po odwróconym indeksie
    scores = np.zeros(n_songs, dtype=np.float32)
    for tag, w in qtw.items():
        if tag not in inv_index:
            continue
        weight = w
        if use_idf and idf_map is not None:
            weight *= float(idf_map.get(tag, 1.0))
        idxs = inv_index[tag]
        scores[idxs] += weight

    # 3) BM25-like normalizacja po długości listy tagów
    if len_norm:
        avg_len = float(song_tag_count.mean())
        avg_len = max(avg_len, 1e-6)
        denom = k1 * ((1.0 - b) + b * (song_tag_count / avg_len)) + 1.0
        scores = scores * ((k1 + 1.0) / denom)

    # 4) alternatywna normalizacja: / sqrt(n_tags)
    if normalize_tags:
        scores = scores / np.sqrt(np.maximum(1.0, song_tag_count))

    return np.clip(scores, 0.0, 1.0)

In [ ]:
scores_songs_tag = score_songs_by_tags(
    query_tag_weights=query_tag_weights,
    inv_index=INV_INDEX,
    song_tag_count=df_songs["tag_count"].to_numpy(dtype=np.float32),
    idf_map=None,
    cfg=TAG_SCORING_CONFIG,
)
scores_songs_tag

array([0.59677625, 0.7975046 , 0.59677625, ..., 0.        , 0.        ,
       0.19989967], dtype=float32)

In [ ]:
def retrieve_candidates_tags(
    scores_songs_tag: np.ndarray,
    n_candidates: int = 400,
    flat_delta: float = 0.05
):
    # bierzemy tylko te, które w ogóle coś dopasowały
    pos_mask = scores_songs_tag > 0.0
    if not np.any(pos_mask):
        return df_songs.iloc[0:0].copy(), q_tw

    pos_idxs = np.where(pos_mask)[0]
    # sortujemy wszystkie dopasowane po score malejąco
    pos_idxs = pos_idxs[np.argsort(-scores_songs_tag[pos_idxs])]

    # jeśli i tak mniej niż n_candidates → bierzemy wszystkie
    if len(pos_idxs) <= n_candidates:
        final_idxs = pos_idxs
    else:
        # score na granicy top-n
        boundary_idx = n_candidates - 1
        boundary_score = float(scores_songs_tag[pos_idxs[boundary_idx]])

        # rozszerzamy cut tak długo, jak kolejne mają ten sam (w praktyce: bardzo podobny) score
        cut = n_candidates
        while (
            cut < len(pos_idxs)
            and abs(float(scores_songs_tag[pos_idxs[cut]]) - boundary_score) <= flat_delta
        ):
            cut += 1

        final_idxs = pos_idxs[:cut]

    candidates = df_songs.iloc[final_idxs].copy()
    candidates["tag_score"] = scores_songs_tag[final_idxs]
    candidates["score"] = candidates["tag_score"]  # spójność z resztą pipeline'u
    return candidates

In [ ]:
candidades_df = retrieve_candidates_tags(scores_songs_tag)
print(candidades_df["track_id"].count())
candidades_df.sample(3)

832


,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,album_images,spotify_id,n_tempo,n_loudness,tags_list,tags_count,tags_embedding,tag_count,tag_score,score
5926,TRXSLRK12903CBE263,What She Came For,Franz Ferdinand,https://p.scdn.co/mp3-preview/d0892e88f705f831...,"rock, alternative, indie, alternative_rock, in...",None,2009,231173,0.826,0.494,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",2NMM2kE37aCzkTzG3VHB01,0.435677,0.867179,"[00s, alternative, alternative rock, british, ...",9,"[0.009147155098617077, 0.022436944767832756, -...",9,0.797505,0.797505
41505,TRPHXBN128F92F9466,Junction,NORA&LEO,https://p.scdn.co/mp3-preview/38bb24e2dc8dd63c...,"rock, electronic, alternative, indie, pop, fem...",None,2010,205031,0.667,0.615,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",66NptAZr7gBa1dZOhksQeH,0.548496,0.849659,"[acoustic, alternative, alternative rock, chil...",36,"[0.00021750800078734756, 0.048592694103717804,...",36,1.000000,1.000000
7232,TRFTFQV128F4233F76,Melatonin,Radiohead,https://p.scdn.co/mp3-preview/3b63f60547e3c2d0...,"electronic, alternative, female_vocalists, alt...",Punk,1998,129400,0.242,0.359,...,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",70lnEv38W5ilDiDdGjgcLP,0.525457,0.730932,"[90s, alternative, alternative rock, ambient, ...",9,"[0.009763325564563274, 0.017561059445142746, -...",9,0.797505,0.797505


In [ ]:
def calculate_audio_match(candidates_df, audio_criteria):
    """
    Liczy dopasowanie kandydatów do kryteriów audio.
    Zwraca wektor (len(candidates),) z wartościami 0.0 - 1.0.
    """
    if candidates_df.empty:
        return np.array([])

    # Jeśli nie ma kryteriów audio, zwracamy 1.0 (neutralny wpływ)
    # lub 0.0 (jeśli chcemy, by audio było wymagane - ale lepiej 1.0)
    if not audio_criteria:
        return np.ones(len(candidates_df))

    # Wektor końcowy (zaczynamy od samych jedynek)
    total_audio_score = np.ones(len(candidates_df))

    # Iterujemy po każdym kryterium (np. energy=0.9, danceability=0.7)
    for feature_name, criteria in audio_criteria:
        target_val = criteria['value']  # np. 0.9

        # Sprawdzamy, czy cecha istnieje w DataFrame
        if feature_name not in candidates_df.columns:
            continue

        # Pobieramy wartości z kolumny (np. [0.4, 0.8, 0.9])
        song_values = candidates_df[feature_name].to_numpy()

        # OBLICZANIE DYSTANSU (Similarity)
        # Prosta metoda liniowa: 1.0 - dystans
        # Jeśli target=0.9, a song=0.5 -> dystans=0.4 -> score=0.6
        dist = np.abs(song_values - target_val)
        sim = 1.0 - dist

        # Opcjonalnie: Boostowanie kary za duże odchylenia (kwadrat)
        # sim = 1.0 - (dist ** 2)

        # Clip, żeby nie wyjść poza zakres (choć przy 0-1 nie powinno)
        sim = np.clip(sim, 0.0, 1.0)

        # Mnożymy (intersekcja warunków): Song musi spełniać Warunek 1 I Warunek 2
        total_audio_score *= sim

    return total_audio_score

In [ ]:
audio_match_vector = calculate_audio_match(candidades_df, criteria_audio)
candidades_df['audio_score'] = audio_match_vector
candidades_df[['artist', 'name', 'tag_score', 'audio_score']].sample(3)

,artist,name,tag_score,audio_score
5238,The Black Keys,Fever,0.797505,0.548766
2458,Panic! at the Disco,Intermission,0.797505,0.263211
2802,Bloc Party,One Month Off,0.797505,0.568464


Fuzja - łączymy wyniki audio i tagów i robimy scoring

In [ ]:
def merge_tag_and_audio_scores(
    candidates_df: pd.DataFrame,
    audio_weight: float = 0.3
) -> pd.DataFrame:
    """
    Łączy wyniki z tagów i audio bezpośrednio w kolumnę 'score'.
    Obie kolumny wejściowe muszą być w zakresie [0, 1].
    """
    df = candidates_df.copy()

    # Sprawdzenie czy mamy wyniki audio
    if 'audio_score' not in df.columns:
        # Brak audio -> Score to po prostu wynik tagowy
        df['score'] = df['tag_score']
    else:
        # Hybryda: Średnia ważona
        # Tag Score (0-1) * (1 - w)  +  Audio Score (0-1) * w
        tag_part = df['tag_score'] * (1.0 - audio_weight)
        audio_part = df['audio_score'] * audio_weight

        df['score'] = tag_part + audio_part

    # Sortowanie
    return df.sort_values('score', ascending=False)

In [ ]:
picked_prompt

'rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii, post rock, alternative rock, rock alternatywny, pop'

In [ ]:
candidades_merged_score_df = merge_tag_and_audio_scores(candidades_df, audio_weight=0.6)
candidades_merged_score_df[['artist', 'name', 'tag_score', 'audio_score', 'score']].head(20)

,artist,name,tag_score,audio_score,score
2418,Bloodhound Gang,Uhn Tiss Uhn Tiss Uhn Tiss,0.797505,0.879300,0.846582
652,Maroon 5,Makes Me Wonder,0.797505,0.859326,0.834597
1735,The Cure,Let's Go to Bed,0.797505,0.835380,0.820230
1567,Gorillaz,Re-Hash,0.797505,0.833930,0.819360
1883,The Stone Roses,Fools Gold,0.797505,0.831040,0.817626
206,Gorillaz,Dirty Harry,0.797505,0.827640,0.815586
256,Smash Mouth,All Star,0.797505,0.813924,0.807356
984,Barenaked Ladies,One Week,0.797505,0.800396,0.799239
27320,Maroon 5,Stutter,0.797505,0.796402,0.796843
5431,Third Eye Blind,Never Let You Go,0.797505,0.795840,0.796506


Dzielimy zestaw kandydatów (piosenek) na trzy poziomy jakości (tzw. tiery) w zależności od wartości dopasowania (score).

Celem tej funkcji jest rozdzielenie wyników wyszukiwania według jakości dopasowania,
tak aby późniejsze etapy
mogły w różny sposób traktować utwory bardzo podobne, średnio podobne i słabe.

Utwory są przypisywane do trzech grup:
  * Tier A — wysokie dopasowanie (`score` >= `t_high`)
  * Tier B — średnie dopasowanie (`t_mid` <= `score` < `t_high`)
  * Tier C — niskie dopasowanie (`score` < `t_mid`)

In [ ]:
def tier_by_score(candidates: pd.DataFrame, t_high: float, t_mid: float):
    """
    Dzieli kandydatów na trzy poziomy jakości (tiery) w zależności od wartości dopasowania (score).

    - Tier A: score >= t_high - bardzo dobre dopasowanie
    - Tier B: t_mid <= score < t_high - średnie dopasowanie
    - Tier C: score < t_mid - słabe dopasowanie

    Args:
        candidates (pd.DataFrame): Dane z kolumną 'score'.
        t_high (float): Próg dla wysokiego dopasowania.
        t_mid (float): Próg dla średniego dopasowania.

    Returns:
        tuple: (tier_a, tier_b, tier_c) — trzy DataFramey z podziałem wg jakości.
    """
    tier_a = candidates[candidates["score"] >= t_high].copy()
    tier_b = candidates[(candidates["score"] < t_high) & (candidates["score"] >= t_mid)].copy()
    tier_c = candidates[candidates["score"] < t_mid].copy()
    return tier_a, tier_b, tier_c

In [ ]:
def calculate_dynamic_thresholds(candidates_df, high_threshold=0.75, mid_threshold=0.5):
    """
    Wylicza progi t_high i t_mid na podstawie tego,
    jak dobre wyniki w ogóle udało się znaleźć.
    """
    if candidates_df.empty:
        return 0.0, 0.0

    # Jaki jest absolutnie najlepszy wynik dla tego zapytania?
    max_score = candidates_df['score'].max()

    # --- LOGIKA ADAPTACYJNA ---

    # 1. Tier A: Musi być blisko lidera (np. max - 0.1),
    #    ALE nie może być mniejszy niż 0.75 (Quality Gate).
    #    Jeśli max_score to 0.5, to t_high będzie 0.75 -> Tier A będzie pusty (i dobrze!)
    t_high = max(high_threshold, max_score - 0.1)

    # 2. Tier B: Musi być sensowny (np. max - 0.3),
    #    ALE nie mniejszy niż 0.4 (żeby nie brać śmieci).
    t_mid = max(mid_threshold, max_score - 0.2)

    return t_high, t_mid

In [ ]:
t_high = np.percentile(scores_songs_tag, 98)
t_mid  = np.percentile(scores_songs_tag, 93)
t_high, t_mid

(np.float32(0.6001277), np.float32(0.59677625))

In [ ]:
t_high, t_mid = calculate_dynamic_thresholds(candidades_merged_score_df)
t_high, t_mid

(0.75, 0.6465818534660339)

In [ ]:
tier_a, tier_b, tier_c = tier_by_score(
    candidades_merged_score_df,
    t_high,
    t_mid
)
print(len(tier_a), len(tier_b), len(tier_c))

34 263 535


 Budowanie zbioru, z którego będziemy losować/wybierać utwory na podstawie dalszych cech (np. popularności).

Funkcja ma na celu stworzenie wystarczająco dużej i jakościowo sensownej puli kandydatów, z którymi będzie dalej pracować algorytm rekomendacji. Nie wybiera jeszcze finalnych piosenek, przygotowuje "roboczy" zestaw do późniejszego ważenia, filtrowania i losowania.


- min_final - mininalna wielkość zbioru, z którego potem będziemy czerpać.
- A - duży score
- B - średni
- C - niski

Logika:
* jeśli A >= min_final -> używamy tylko A
* elif A+B >= min_final -> używamy A + B
* else -> A + B + kawałek C

In [ ]:
def build_working_set(
    tier_a: pd.DataFrame,
    tier_b: pd.DataFrame,
    tier_c: pd.DataFrame,
    target_pool_size: int,
    min_required_size: int,
    popularity_rescue_ratio: float = 0.2
) -> pd.DataFrame:
    """
    Buduje zbiór roboczy.
    Ważne: Przy dobieraniu z Tier B, dba o to, aby nie wziąć tylko niszowych utworów z dobrym score,
    ale też "uratować" popularne utwory, które mają przyzwoity score (są w Tier B).
    Args:
        tier_a (pd.DataFrame): Utwory o wysokim dopasowaniu (Hity jakościowe).
                               Bierzemy je zawsze w całości.
        tier_b (pd.DataFrame): Utwory o średnim dopasowaniu (Dobre, ale nie idealne).
                               Służą do uzupełnienia puli do `target_pool_size`.
        tier_c (pd.DataFrame): Utwory o niskim dopasowaniu (Słabe).
                               Używane tylko awaryjnie, jeśli nie osiągniemy `min_required_size`.
        target_pool_size (int): Docelowa wielkość zbioru roboczego (np. 50-100).
                                Tyle kandydatów chcemy dać algorytmowi losującemu,
                                aby miał swobodę w dobieraniu High/Mid/Low popularity.
        min_required_size (int): ABSOLUTNE MINIMUM. Tyle piosenek ma mieć finalna playlista.
                                 Jeśli A+B to za mało, dobierzemy stąd resztę z Tier C.
                                 Zalecane: tyle, ile user zażądał (np. 15).
        popularity_rescue_ratio (float): Jaka część slotów z Tieru B ma być zarezerwowana
                                         dla utworów NAJPOPULARNIEJSZYCH (zamiast tych z najlepszym score).
                                         Np. 0.3 oznacza, że 30% dobieranych utworów z B to hity,
                                         a 70% to najlepiej pasujące niszowe.
    """

    # 1. ZAWSZE bierzemy całe Tier A
    working_parts = [tier_a]
    current_count = len(tier_a)

    # 2. UZUPEŁNIANIE Z TIER B
    if current_count < target_pool_size:
        needed = target_pool_size - current_count

        if len(tier_b) <= needed:
            working_parts.append(tier_b)
            current_count += len(tier_b)
        else:
            n_pop = int(needed * popularity_rescue_ratio)
            n_score = needed - n_pop

            b_pop_rescued = tier_b.sort_values("popularity", ascending=False).head(n_pop)
            remaining_b = tier_b.drop(b_pop_rescued.index)
            b_score_top = remaining_b.sort_values("score", ascending=False).head(n_score)

            working_parts.extend([b_pop_rescued, b_score_top])
            current_count += (len(b_pop_rescued) + len(b_score_top))

    # 3. RATUNEK Z TIER C
    if current_count < min_required_size:
        needed = min_required_size - current_count
        part_c = tier_c.sort_values("score", ascending=False).head(needed)
        working_parts.append(part_c)

    if not working_parts:
        return pd.DataFrame(columns=tier_a.columns)

    # 4. Sklejanie
    working = pd.concat(working_parts, ignore_index=True) # Reset indexu przy sklejaniu

    # === POPRAWKA: BEZPIECZNE USUWANIE DUPLIKATÓW ===

    # Ustalamy po czym identyfikujemy piosenkę.
    # Unikamy sprawdzania kolumn zawierających listy (jak tags_list).
    subset_cols = []

    if 'id' in working.columns:
        subset_cols = ['id']
    elif 'name' in working.columns and 'artist' in working.columns:
        subset_cols = ['name', 'artist']

    if subset_cols:
        working = working.drop_duplicates(subset=subset_cols)
    else:
        # Ostateczność: Jeśli nie mamy ID ani Nazwy, a mamy unikalny index w piosenkach
        # to po prostu nie robimy drop_duplicates() po zawartości,
        # bo concat z ignore_index=True i tak stworzył nowy indeks.
        # Ewentualnie, jeśli masz pewność, że w indexach źródłowych (tier_a/b/c)
        # były ID piosenek, to nie używaj ignore_index=True w concat.
        pass

    # Finalne sortowanie
    working = working.sort_values("score", ascending=False).reset_index(drop=True)

    return working

In [ ]:
working = build_working_set(tier_a, tier_b, tier_c, SCORE_TIERS_CONFIG['min_final'], SCORE_TIERS_CONFIG['max_c_from_low_tier'])
# working
working["score"].value_counts()

,count
score,
0.846582,1
0.834597,1
0.820230,1
0.819360,1
0.817626,1
...,...
0.688787,1
0.687797,1
0.682413,1


Podział kandydatów wg popularności - chcemy uzyskać część popularnych utworów, część niszowych.

In [ ]:
def bucket_by_popularity(working: pd.DataFrame, p_high: int, p_mid: int):
    """
    Dzieli utwory na trzy grupy (bucket'y) według ich popularności.

    - High:  popularity >= p_high
    - Mid:   p_mid <= popularity < p_high
    - Low:   popularity < p_mid

    Args:
        working (pd.DataFrame): Zbiór roboczy utworów z kolumną 'popularity'.
        p_high (int): Próg dla wysokiej popularności.
        p_mid (int): Próg dla średniej popularności.

    Returns:
        tuple: (pop_high, pop_mid, pop_low) — trzy DataFrame'y z podziałem wg popularności.
    """

    pop_high = working[working["popularity"] >= p_high].copy()
    pop_mid = working[(working["popularity"] < p_high) & (working["popularity"] >= p_mid)].copy()
    pop_low = working[working["popularity"] < p_mid].copy()
    return pop_high, pop_mid, pop_low

In [ ]:
pop_high, pop_mid, pop_low = bucket_by_popularity(working, POPULARITY_CONFIG['p_high'], POPULARITY_CONFIG['p_mid'])
pop_high

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,spotify_id,n_tempo,n_loudness,tags_list,tags_count,tags_embedding,tag_count,tag_score,score,audio_score
5,TRBENMR12903CE9B16,Dirty Harry,Gorillaz,https://p.scdn.co/mp3-preview/7409d884124b8379...,"rock, electronic, alternative, indie, pop, alt...",None,2005,223800,0.755,0.868,...,2bfGNzdiRa1jXZRdfssSzR,0.402629,0.831071,"[00s, alternative, alternative rock, british, ...",12,"[-0.004335538484156132, 0.02139289304614067, -...",12,0.797505,0.815586,0.827640
6,TRDBTTU128F931E20E,All Star,Smash Mouth,https://p.scdn.co/mp3-preview/24a26524cd1e3641...,"rock, alternative, pop, alternative_rock, soun...",None,2000,198933,0.728,0.883,...,3cfOd4CMv2snFaKAnMdnvK,0.435430,0.846972,"[90s, alternative, alternative rock, pop, pop ...",8,"[0.010836097411811352, 0.020900528877973557, -...",8,0.797505,0.807356,0.813924
12,TRYJPIH128F4262602,This Love,Maroon 5,https://p.scdn.co/mp3-preview/86ee8e73aa2c439b...,"rock, alternative, pop, alternative_rock, love...",RnB,2007,205786,0.714,0.862,...,6ECp64rv50XVz93WvxXMGF,0.397823,0.870447,"[00s, alternative, alternative rock, love, pop...",7,"[-0.0067707085981965065, 0.013502966612577438,...",7,0.797505,0.788843,0.783068
14,TRVIOPH128E079894E,Walkin' On The Sun,Smash Mouth,https://p.scdn.co/mp3-preview/7c66b4e4589eb38c...,"rock, alternative, pop, alternative_rock, 90s,...",Rock,2010,206133,0.736,0.975,...,4TQeNHx85xcP9YRkvJC6K1,0.516164,0.863172,"[90s, alternative, alternative rock, pop, pop ...",7,"[0.007887734100222588, 0.025863399729132652, -...",7,0.797505,0.782982,0.773300
17,TRUMEFQ128F92CA67B,Believer,Imagine Dragons,https://p.scdn.co/mp3-preview/a14b2a107bcd428e...,"rock, alternative, indie, pop, alternative_roc...",None,2017,203782,0.772,0.775,...,0pqnGHJpmpxLKifKRmU6WP,0.523150,0.873825,"[alternative, alternative rock, american, indi...",10,"[0.0006847615004517138, 0.014685671776533127, ...",10,0.797505,0.776802,0.763000
18,TRXCBZS128F92FA4CA,Butterfly,Crazy Town,https://p.scdn.co/mp3-preview/8bab4427d8ce0fcb...,"rock, alternative, pop, alternative_rock, rap,...",Pop,2000,215800,0.739,0.808,...,4BggEwLhGfrbrl7JBhC8EC,0.433287,0.867399,"[90s, alternative, alternative rock, hip hop, ...",8,"[0.00837296899408102, 0.02851223386824131, -0....",8,0.797505,0.776089,0.761812
19,TRCTVFN128F148D0FD,The Bad Touch,Bloodhound Gang,https://p.scdn.co/mp3-preview/8e9f536bcb9faf0f...,"rock, alternative, pop, alternative_rock, punk...",None,2013,260506,0.829,0.712,...,5EYdTPdJD74r9EVZBztqGG,0.514795,0.844662,"[90s, alternative, alternative rock, dance, po...",7,"[0.004780516028404236, 0.02183462493121624, -0...",7,0.797505,0.771611,0.754348
20,TRIMVXU128F4275E4A,Girls & Boys,Blur,https://p.scdn.co/mp3-preview/567ec258bd4efde2...,"rock, alternative, indie, pop, alternative_roc...",Electronic,1994,290333,0.691,0.853,...,5CeL9C3bsoe4yzYS1Qz8cw,0.502543,0.765249,"[90s, alternative, alternative rock, british, ...",9,"[0.009572041220963001, 0.025483649224042892, -...",9,0.797505,0.771296,0.753823
21,TRJFMNK128E07840C2,19-2000,Gorillaz,https://p.scdn.co/mp3-preview/7cabceed47ab219f...,"rock, electronic, alternative, indie, pop, alt...",Rock,2001,210880,0.815,0.720,...,71koAFCnGnugRdGIDfS7f4,0.376375,0.827897,"[00s, alternative, alternative rock, british, ...",11,"[0.005996811203658581, 0.03837985917925835, -0...",11,0.797505,0.769182,0.750300
22,TRYEZGE128F92D04C8,Feel Good Inc.,Gorillaz,https://p.scdn.co/mp3-preview/7388c425022ced92...,"rock, electronic, alternative, indie, pop, alt...",RnB,2010,223106,0.809,0.720,...,0d28khcov6AiegSCpG5TuT,0.579987,0.820590,"[alternative, alternative rock, british, dance...",10,"[-0.0006239848444238305, 0.02780434861779213, ...",10,0.797505,0.766230,0.745380


In [ ]:
def weighted_sample(df: pd.DataFrame, k: int, alpha: float):
    """
    Losuje do k wierszy z df bez zwracania, z wagami ~ score^alpha.
    Jak df ma mniej niż k wierszy → zwraca wszystkie.
    """
    if k <= 0 or len(df) == 0:
        return df.iloc[0:0]

    k = min(k, len(df))

    scores = df["score"].to_numpy(dtype=np.float32)
    scores = np.clip(scores, 1e-9, None)
    w = np.power(scores, alpha)
    w_sum = w.sum()
    if w_sum <= 0:
        w = np.full_like(w, 1.0 / len(w), dtype=np.float32)
    else:
        w /= w_sum

    idx = np.random.choice(len(df), size=k, replace=False, p=w)
    return df.iloc[idx]

In [ ]:
# def sample_final_songs(
#     working: pd.DataFrame,
#     popularity_cfg: dict,
#     sampling_cfg: dict,
# ) -> pd.DataFrame:
#     """
#     Prosta wersja:
#     - dzieli na high / mid / low po popularności,
#     - dodaje forced_popular,
#     - resztę dobiera z bucketów high/mid/low wg miksu i losowania score^alpha.
#     """

#     if len(working) == 0:
#         return working.iloc[0:0].copy()

#     final_n = sampling_cfg.get("final_n", 15)
#     alpha   = sampling_cfg.get("alpha", 2.0)

#     p_high = popularity_cfg.get("p_high", 70)
#     p_mid  = popularity_cfg.get("p_mid", 35)
#     mix    = popularity_cfg.get("mix", {"high": 0.4, "mid": 0.35, "low": 0.25})

#     forced_popular     = popularity_cfg.get("forced_popular", 0)
#     forced_popular_min = popularity_cfg.get("forced_popular_min", p_high)

#     # 1) Bucketowanie po popularności
#     pop_high, pop_mid, pop_low = bucket_by_popularity(working, p_high=p_high, p_mid=p_mid)

#     final_parts = []

#     ## TODO: to nie miało być top N najpopularniejsze! musimy tutaj dac element losowosci.
#     # 2) Forced popular – weź top N najbardziej popularnych o największym score
#     forced_pool = pop_high[pop_high["popularity"] >= forced_popular_min].copy()
#     forced_pool = forced_pool.sort_values(["score", "popularity"], ascending=False)

#     forced_taken = forced_pool.head(forced_popular)
#     final_parts.append(forced_taken)

#     # usuń je z bucketów, żeby nie dublować
#     used_idx = set(forced_taken.index)
#     pop_high = pop_high[~pop_high.index.isin(used_idx)]
#     pop_mid  = pop_mid[~pop_mid.index.isin(used_idx)]
#     pop_low  = pop_low[~pop_low.index.isin(used_idx)]

#     n_forced = len(forced_taken)
#     remaining = max(0, final_n - n_forced)

#     if remaining == 0:
#         combined = pd.concat(final_parts, ignore_index=False)
#         return combined.sort_values("score", ascending=False).reset_index(drop=True)

#     # 3) policz ile slotów ma iść do którego bucketu
#     target_high = int(round(remaining * mix.get("high", 0.0)))
#     target_mid  = int(round(remaining * mix.get("mid",  0.0)))
#     target_low  = remaining - target_high - target_mid  # reszta do low

#     # prosty clamp, jeśli w bucketach jest mniej niż target
#     target_high = min(target_high, len(pop_high))
#     target_mid  = min(target_mid, len(pop_mid))
#     target_low  = min(target_low, len(pop_low))

#     # 4) losujemy z bucketów wg score^alpha
#     sampled_high = weighted_sample(pop_high, target_high, alpha)
#     sampled_mid  = weighted_sample(pop_mid,  target_mid,  alpha)
#     sampled_low  = weighted_sample(pop_low,  target_low,  alpha)

#     final_parts.extend([sampled_high, sampled_mid, sampled_low])

#     combined = pd.concat(final_parts, ignore_index=False)

#     # 5) Jeśli dalej brakuje do final_n → dobierz z reszty working setu po score^alpha
#     if len(combined) < final_n:
#         missing = final_n - len(combined)
#         used_idx = set(combined.index)
#         remaining_pool = working[~working.index.isin(used_idx)]
#         extra = weighted_sample(remaining_pool, missing, alpha)
#         combined = pd.concat([combined, extra], ignore_index=False)

#     # 6) FINALNE TASOWANIE
#     # frac=1 oznacza "weź 100% wierszy, ale w losowej kolejności"
#     do_shuffle = sampling_cfg.get("shuffle", True)
#     if do_shuffle:
#         combined = combined.sample(frac=1).reset_index(drop=True)
#     else:
#         # Opcja dla debugowania: najlepsze na górze
#         combined = combined.sort_values("score", ascending=False).reset_index(drop=True)

#     return combined


In [ ]:
def sample_final_songs(
    working: pd.DataFrame,
    popularity_cfg: dict,
    sampling_cfg: dict,
) -> pd.DataFrame:
    """
    Wybiera finalną playlistę z puli roboczej (working set).

    Logika:
    1. Dzieli utwory na High/Mid/Low Popularity.
    2. "Forced Popular": Wybiera N hitów (jeśli skonfigurowano),
       ale losuje je ważonym scorem, żeby nie grać w kółko tego samego.
    3. Resztę playlisty dobiera z bucketów High/Mid/Low wg zadanego miksu (np. 40%/40%/20%).

    Dzięki temu, że 'score' zawiera już ocenę Audio, losowanie naturalnie
    preferuje utwory pasujące brzmieniowo.
    """

    if len(working) == 0:
        return working.iloc[0:0].copy()

    # Konfiguracja
    final_n = sampling_cfg.get("final_n", 15)
    alpha   = sampling_cfg.get("alpha", 2.0) # Siła wpływu score na losowanie

    p_high = popularity_cfg.get("p_high", 70)
    p_mid  = popularity_cfg.get("p_mid", 35)
    mix    = popularity_cfg.get("mix", {"high": 0.4, "mid": 0.35, "low": 0.25})

    forced_popular         = popularity_cfg.get("forced_popular", 0)
    forced_popular_min     = popularity_cfg.get("forced_popular_min", p_high)

    # 1) Bucketowanie po popularności (korzystamy z Twojej funkcji pomocniczej)
    pop_high, pop_mid, pop_low = bucket_by_popularity(working, p_high=p_high, p_mid=p_mid)

    final_parts = []

    # 2) FORCED POPULAR (Zmienione: Losowość zamiast sztywnego .head())
    # Wybieramy pulę bardzo popularnych utworów (powyżej forced_popular_min)
    # Zazwyczaj to podzbiór pop_high
    forced_pool = working[working["popularity"] >= forced_popular_min].copy()

    # Zamiast brać top N na sztywno, losujemy z nich, preferując te z wyższym score.
    # Dzięki temu playlista jest bardziej różnorodna przy kolejnych wywołaniach.
    forced_taken = weighted_sample(forced_pool, forced_popular, alpha)

    final_parts.append(forced_taken)

    # Usuwamy wybrane utwory z bucketów, żeby ich nie dublować
    used_idx = set(forced_taken.index)
    pop_high = pop_high[~pop_high.index.isin(used_idx)]
    pop_mid  = pop_mid[~pop_mid.index.isin(used_idx)]
    pop_low  = pop_low[~pop_low.index.isin(used_idx)]

    # 3) Obliczamy ile slotów zostało do wypełnienia
    n_forced = len(forced_taken)
    remaining = max(0, final_n - n_forced)

    if remaining == 0:
        combined = pd.concat(final_parts, ignore_index=False)
        return combined.sort_values("score", ascending=False).reset_index(drop=True)

    # 4) Wyliczamy cele dla bucketów (Mix)
    target_high = int(round(remaining * mix.get("high", 0.0)))
    target_mid  = int(round(remaining * mix.get("mid",  0.0)))
    target_low  = remaining - target_high - target_mid  # Reszta do low

    # Clamp (nie możemy wziąć więcej niż jest w buckecie)
    target_high = min(target_high, len(pop_high))
    target_mid  = min(target_mid, len(pop_mid))
    target_low  = min(target_low, len(pop_low))

    # 5) Główne losowanie z bucketów (Weighted Sample wg Score)
    # Tu dzieje się magia Audio/Tagów: piosenki, które lepiej pasują (wyższy score),
    # mają większą szansę na wylosowanie.
    sampled_high = weighted_sample(pop_high, target_high, alpha)
    sampled_mid  = weighted_sample(pop_mid,  target_mid,  alpha)
    sampled_low  = weighted_sample(pop_low,  target_low,  alpha)

    final_parts.extend([sampled_high, sampled_mid, sampled_low])

    combined = pd.concat(final_parts, ignore_index=False)

    # 6) Fill Gaps (Jeśli zaokrąglenia albo braki w bucketach sprawiły, że mamy za mało)
    if len(combined) < final_n:
        missing = final_n - len(combined)
        used_idx = set(combined.index)

        # Bierzemy resztki z całego working setu
        remaining_pool = working[~working.index.isin(used_idx)]
        extra = weighted_sample(remaining_pool, missing, alpha)

        combined = pd.concat([combined, extra], ignore_index=False)

    # 6) FINALNE TASOWANIE
    # frac=1 oznacza "weź 100% wierszy, ale w losowej kolejności"
    do_shuffle = sampling_cfg.get("shuffle", True)
    if do_shuffle:
        combined = combined.sample(frac=1).reset_index(drop=True)
    else:
        # Opcja dla debugowania: najlepsze na górze
        combined = combined.sort_values("score", ascending=False).reset_index(drop=True)

    return combined


In [ ]:
sample_final_songs(working, POPULARITY_CONFIG, SAMPLING_CONFIG)[["name", "artist", "tags_list", "popularity", "score"]]

,name,artist,tags_list,popularity,score
0,Teenage Dirtbag,Wheatus,"[00s, 90s, alternative, alternative rock, indi...",82.0,0.703170
1,This Love,Maroon 5,"[00s, alternative, alternative rock, love, pop...",85.0,0.788843
2,Ride,Twenty One Pilots,"[alternative, alternative rock, american, danc...",82.0,0.682413
3,15 Step,Radiohead,"[alternative, alternative rock, british, britp...",67.0,0.719262
4,Positive Tension,Bloc Party,"[alternative, alternative rock, british, britp...",43.0,0.731751
5,Never Let You Go,Third Eye Blind,"[90s, alternative, alternative rock, pop, pop ...",69.0,0.796506
6,Uhn Tiss Uhn Tiss Uhn Tiss,Bloodhound Gang,"[alternative, alternative rock, dance, electro...",57.0,0.846582
7,Clear Skies,Keane,"[alternative, alternative rock, british, britp...",27.0,0.765836
8,Point of No Return,Stasis,"[alternative, alternative rock, ambient, ameri...",5.0,0.784535
9,Beverly Hills,Weezer,"[00s, 90s, alternative, alternative rock, amer...",70.0,0.746595
